In [139]:
from pyspark import SparkConf, SparkContext
import os
from datetime import datetime, timedelta
import pandas as pd

from pyspark.sql import SQLContext
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, StringType, DateType, TimestampType, BooleanType

In [ ]:
inputDir, outputDir = '/Users/sofiapetryshyn/Desktop/bigdata/hw3/input_data', \
                    '/Users/sofiapetryshyn/Desktop/bigdata/hw3/output'

In [60]:
one_file_df = pd.read_csv(os.path.join(inputDir, \
                                       list(filter(lambda file: file.endswith('csv'), os.listdir(inputDir)))[0]))
one_file_df.head()


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,SbOwzAl9ZfQ,17.14.11,Capítulo 12 | MasterChef 2017,MasterChef 2017,24,2017-11-13T06:06:22.000Z,"MasterChef Junior 2017|""TV Azteca""|""recetas""|""...",310130,4182,361,1836,https://i.ytimg.com/vi/SbOwzAl9ZfQ/default.jpg,False,False,False,Disfruta la presencia del Chef Torreblanca en ...
1,klOV6Xh-DnI,17.14.11,ALEXA EX-INTEGRANTE DEL GRUPO TIMBIRICHE RENUN...,Micky Contreras Martinez,22,2017-11-13T05:11:58.000Z,La Voz Mexico 7,104972,271,174,369,https://i.ytimg.com/vi/klOV6Xh-DnI/default.jpg,False,False,False,ALEXA EX-INTEGRANTE DEL GRUPO TIMBIRICHE RENUN...
2,6L2ZF7Qzsbk,17.14.11,LOUIS CKAGÓ - EL PULSO DE LA REPÚBLICA,El Pulso De La República,25,2017-11-13T17:00:02.000Z,"Chumel Torres|""El Pulso de la Republica""|""noti...",136064,10105,266,607,https://i.ytimg.com/vi/6L2ZF7Qzsbk/default.jpg,False,False,False,La canción del principio se llama “Este espíri...
3,hcY52MFWMDM,17.14.11,Sismo de 6.7 sacude Costa Rica 12 Noviembre 2017,Casanare,25,2017-11-13T03:47:10.000Z,"temblor|""costa rica""|""sismo en costa rica""",96153,378,171,208,https://i.ytimg.com/vi/hcY52MFWMDM/default.jpg,False,False,False,El video es de un Walmart en el pais centroame...
4,_OXDcGPVAa4,17.14.11,DOG HACKS | MUSAS LESSLIE LOS POLINESIOS,Musas,26,2017-11-13T19:17:48.000Z,"MUSAS|""lesslie""|""karen""|""hacks""|""perros""|""dogs...",499965,57781,681,7428,https://i.ytimg.com/vi/_OXDcGPVAa4/default.jpg,False,False,False,MI HERMANO NARRA MI RUTINA DE MAQUILLAJE\nhttp...


### Task 1
Find Top 10 videos that were amongst the trending videos for the highest number of days (it doesn't need to be a consecutive period of time). You should also include information about different metrics for each day the video was trending.

In [79]:
res_dict = {}

df_unique = one_file_df.groupby('video_id')['trending_date'].nunique()\
    .reset_index().sort_values(by=['trending_date'], ascending=False).reset_index(drop=True)
df_unique = df_unique.loc[df_unique.video_id != '#NAME?'][:10]
print(df_unique)

       video_id  trending_date
1   siphkFpHQhs              4
2   ejhNHlFuhz0              4
3   xTlNMmZKwpA              4
4   FRMw8hiphrc              4
5   FVRPzwb1Vi4              4
6   ezcKDkH1OZc              4
7   tLOl3s0uIM4              4
8   FlsCjmMhFmw              4
9   ZRx76gi8aUA              4
10  mIL-eTj5udg              4


In [104]:
import json

In [116]:
# For one region
task = 1
for file in list(filter(lambda file: file.endswith('csv'), os.listdir(inputDir))):
    country = file[:2]
    print(f'Country: {country}')
    file = os.path.join(inputDir, file)
    one_file_df = pd.read_csv(file)
    
    df_unique = one_file_df.groupby('video_id')['trending_date'].nunique()\
                .reset_index().sort_values(by=['trending_date'], ascending=False).reset_index(drop=True)
    df_unique = df_unique.loc[df_unique.video_id != '#NAME?'][:10]
    
    video_arr = []
    # For one trending video
    for video_id in df_unique.video_id:
        tmp_dict = {}
        tmp_dict['id'] = str(video_id)
        tmp_series = one_file_df.loc[one_file_df.video_id == video_id].head(1)
        tmp_dict['title'] = str(tmp_series.title.values[0])
        tmp_dict['description'] = str(tmp_series.description.values[0])
        tmp_dict['latest_views'] = int(tmp_series.views.values[0])
        tmp_dict['latest_likes'] = int(tmp_series.likes.values[0])
        tmp_dict['latest_dislikes'] = int(tmp_series.dislikes.values[0])
        tmp_dict['trending_days'] = [
            {
                'date': str(series[1].trending_date),
                'views': int(series[1].views),
                'likes': int(series[1].likes),
                'dislikes': int(series[1].dislikes),
            }
            for series in one_file_df.loc[one_file_df.video_id == video_id].iterrows()
        ]

        video_arr.append(tmp_dict)


    res_dict = {
        'videos': video_arr
    }
    cur_dir = f'./output/copycat_inc/{task}/{country}'
    os.makedirs(cur_dir)
    out_file = open(os.path.join(cur_dir, 'result.json'), "w+")
    json.dump(res_dict, out_file, indent=4)
    out_file.close()
    

Country: MX
Country: IN
Country: DE
Country: JP
Country: KR
Country: CA
Country: RU
Country: FR
Country: US
Country: GB


### Task 2
Find what was the most popular category for each week (7 days slices). Popularity is decided based on the total number of views for videos of this category. Note, to calculate it you can’t just sum up the number of views. If a particular video appeared only once during the given period, it shouldn’t be counted. Only if it appeared more than once you should count the number of new views. For example, if video A appeared on day 1 with 100 views, then on day 4 with 250 views and again on day 6 with 400 views, you should count it as 400 - 100 = 300. For our purpose, it will mean that this particular video was watched 300 times in the given time period.

In [143]:
one_file_df.head(2)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,Jw1Y-zhQURU,17.14.11,John Lewis Christmas Ad 2017 - #MozTheMonster,John Lewis,26,2017-11-10T07:38:29.000Z,"christmas|""john lewis christmas""|""john lewis""|...",7224515,55681,10247,9479,https://i.ytimg.com/vi/Jw1Y-zhQURU/default.jpg,False,False,False,Click here to continue the story and make your...
1,3s1rvMFUweQ,17.14.11,Taylor Swift: …Ready for It? (Live) - SNL,Saturday Night Live,24,2017-11-12T06:24:44.000Z,"SNL|""Saturday Night Live""|""SNL Season 43""|""Epi...",1053632,25561,2294,2757,https://i.ytimg.com/vi/3s1rvMFUweQ/default.jpg,False,False,False,Musical guest Taylor Swift performs …Ready for...


In [166]:
import operator

In [169]:
task = 2
for file in list(filter(lambda file: file.endswith('csv'), os.listdir(inputDir))):
    country = file[:2]
    print(f'Country: {country}')
    file = os.path.join(inputDir, file)
    one_file_df = pd.read_csv(file)
    one_file_df.trending_date = list(map(lambda date_: datetime.strptime('20' + date_, '%Y.%d.%m'), \
                                         one_file_df.trending_date))
    one_file_df = one_file_df.loc[one_file_df.video_id != '#NAME?']
    start_d = one_file_df.trending_date.min()
    res_dict = {
        'weeks': []
    }
    # For each week
    while start_d <= one_file_df.trending_date.max():
        end_period = start_d + timedelta(days=7)
        tmp_dict = {}
        tmp_dict['start_date'] = str(start_d)
        tmp_dict['end_date'] = str(end_period)
        week_df = one_file_df.loc[(start_d <= one_file_df.trending_date) & (one_file_df.trending_date < end_period)]
        v = week_df.video_id.value_counts()
        week_df = week_df[week_df.video_id.isin(v.index[v.gt(2)])]
        
        category_dict = {}
        category_videos_dict = {}
        # For all the video occurances
        for video_id in set(week_df.video_id):
            week_video_df = week_df.loc[(week_df.video_id == video_id)]
            num_of_views_during_the_week = week_video_df.tail(1).\
                                            views.values[0] - week_video_df.head(1).views.values[0]

            cur_category = week_video_df.head(1).category_id.values[0]
            
            if category_videos_dict.get(cur_category, None) is None:
                category_videos_dict[cur_category] = []
            category_videos_dict[cur_category].append(str(week_video_df.head(1).video_id.values[0]))
            
            if category_dict.get(cur_category, None) is None:
                category_dict[cur_category] = 0
            category_dict[cur_category] += num_of_views_during_the_week

        max_category_id = max(category_dict.items(), key=operator.itemgetter(1))[0]
        tmp_dict['category_id'] = int(max_category_id)
        tmp_dict['number_of_videos'] = int(len(category_videos_dict[max_category_id]))
        tmp_dict['total_views'] = int(category_dict[max_category_id])
        tmp_dict['video_ids'] = category_videos_dict[cur_category]
        # each week dictionsry
        res_dict['weeks'].append(tmp_dict)
        start_d = end_period
    
    cur_dir = f'./output/copycat_inc/{task}/{country}'
    os.makedirs(cur_dir)
    out_file = open(os.path.join(cur_dir, 'result.json'), "w+")
    json.dump(res_dict, out_file, indent=4)
    out_file.close()
    

Country: MX
Country: IN
Country: DE
Country: JP
Country: KR
Country: CA
Country: RU
Country: FR
Country: US
Country: GB


### Task 3
What were the 10 most used tags amongst trending videos for each 30days time period? Note, if during the specified period the same video appears multiple times, you should count tags related to that video only once.

In [170]:
df.head(1)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,Jw1Y-zhQURU,2017-11-14,John Lewis Christmas Ad 2017 - #MozTheMonster,John Lewis,26,2017-11-10T07:38:29.000Z,"christmas|""john lewis christmas""|""john lewis""|...",7224515,55681,10247,9479,https://i.ytimg.com/vi/Jw1Y-zhQURU/default.jpg,False,False,False,Click here to continue the story and make your...


In [182]:
from collections import Counter

In [230]:
task = 3
for file in list(filter(lambda file: file.endswith('csv'), os.listdir(inputDir))):
    country = file[:2]
    print(f'Country: {country}')
    file = os.path.join(inputDir, file)
    one_file_df = pd.read_csv(file)
    one_file_df.trending_date = list(map(lambda date_: datetime.strptime('20' + date_, '%Y.%d.%m'), \
                                         one_file_df.trending_date))
    one_file_df = one_file_df.loc[one_file_df.video_id != '#NAME?']
    start_d = one_file_df.trending_date.min()
    res_dict = {
        'months': []
    }
    
    # For each month
    while start_d <= one_file_df.trending_date.max():
        end_period = start_d + timedelta(days=30)
        tmp_dict = {}
        tmp_dict['start_date'] = str(start_d)
        tmp_dict['end_date'] = str(end_period)
        month_df = one_file_df.loc[(start_d <= one_file_df.trending_date) & (one_file_df.trending_date < end_period)]
        month_df = month_df.drop_duplicates(subset=['video_id'])
        
        month_tags = []
        for video_row in month_df.iterrows():
            month_tags += list(filter(lambda tag: tag != '[none]', video_row[1].tags.replace("'", "").replace("\"", "").split('|')))
        month_tags_counter = Counter(month_tags)
        top_10_tags_counts = month_tags_counter.most_common(10)
        top_10_tags = [el[0] for el in top_10_tags_counts]
        tmp_dict['tags'] = []
        
        for cur_tag in top_10_tags:
            mask = month_df.tags.apply(lambda x: any(tag for tag in [cur_tag] if tag in x.replace("'", "").replace("\"", "").split('|')))
            month_videos_with_tags = [video for video in month_df[mask].video_id.values]
            
            tmp_dict['tags'].append(
                {
                    'tag': cur_tag,
                    'number_of_videos': len(month_videos_with_tags),
                    'video_ids': month_videos_with_tags
                }
            )
#         print(tmp_dict)
        res_dict['months'].append(tmp_dict)        
        start_d = end_period
    
    cur_dir = f'./output/copycat_inc/{task}/{country}'
    os.makedirs(cur_dir)
    out_file = open(os.path.join(cur_dir, 'result.json'), "w+")
    json.dump(res_dict, out_file, indent=4)
    out_file.close()


Country: MX
Country: IN
Country: DE
Country: JP
Country: KR
Country: CA
Country: RU
Country: FR
Country: US
Country: GB


### Task 4
Show the top 20 channels by the number of views for the whole period. Note, if there are multiple appearances of the same video for some channel, you should take into account only the last appearance (with the highest number of views).

In [27]:
# list(one_file_df.columns)

In [29]:
conf = SparkConf().setMaster('local')
sc = SparkContext(conf=conf)

In [35]:
sqlContext = SQLContext(sc)

In [39]:
schemaDf = StructType([
    StructField("video_id", StringType()),
    StructField("trending_date", DateType()),
    StructField("title", StringType()),
    StructField("channel_title", StringType()),
    StructField("category_id", IntegerType()),
    StructField("publish_time", TimestampType()),
    StructField("tags", StringType()),
    StructField("views", IntegerType()),
    StructField("likes", IntegerType()),
    StructField("dislikes", IntegerType()),
    StructField("comment_count", IntegerType()),
    StructField("thumbnail_link", StringType()),
    StructField("comments_disabled", BooleanType()),
    StructField("ratings_disabled", BooleanType()),
    StructField("video_error_or_removed", BooleanType()),
    StructField("description", StringType())
])

In [59]:
for file in list(filter(lambda file: file.endswith('csv'), os.listdir(inputDir))):
    file = os.path.join(inputDir, file)
    df = sqlContext.read \
        .format('com.databricks.spark.csv') \
        .options(header='true') \
        .load(file, schema = schemaDf)
    
#     print(df.select('video_id', 'trending_date').collect())
#     break
    
    print(df.select('video_id', 'trending_date').groupBy('video_id').count().show())
    break
    
#     o_file = os.path.join(outputDir, 'newcars.csv')
#     df.select('video_id', 'trending_date').write \
#     .format('com.databricks.spark.csv') \
#     .save(o_file)

    

+-----------+-----+
|   video_id|count|
+-----------+-----+
|1q41CjSKa7s|    1|
|VvsVDp0aYHY|    1|
|3W9uwoM3WAc|    1|
|vDsDg7iuT-A|    1|
|MsRWkjuhblg|    1|
|hM23vXlT_VY|    1|
|5tHyEZvqbpE|    1|
|P7MJnELbCIk|    1|
|8NXpfS2wSic|    1|
|JFMVjX5Jexo|    1|
|R7rNcvcM5Ag|    2|
|dSZTHvARYPg|    1|
|R420j7DG_Ik|    2|
|mGwuSNAVvTc|    1|
|pKMTAU14-7s|    1|
|42KMIRozQcM|    2|
|173_sjiKXQk|    1|
|G_Ar-vHo10Q|    1|
|B8_7vu1Akpk|    1|
|VX1geS-D3oU|    2|
+-----------+-----+
only showing top 20 rows

None


In [3]:
rdd = sc.parallelize(range(10000))

In [4]:
rdd.count()

10000

In [5]:
sc

<SparkContext master=local[*] appName=pyspark-shell>